In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://metro.cdmx.gob.mx/longitud-de-estacion"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tabla = soup.find('table')
if not tabla:
    raise ValueError("Tabla no encontrada")

filas = tabla.find_all('tr')[1:]  # Omitir encabezado
linea_actual = None
datos = []

for fila in filas:
    celdas = fila.find_all('td')
    if not celdas:
        continue

    # Detectar nueva línea (primera celda con rowspan o contenido numérico)
    if celdas[0].get_text(strip=True).isdigit():
        linea_actual = celdas[0].get_text(strip=True)
        # Extraer datos de la primera estación en esta misma fila
        if len(celdas) >= 4:
            tramo = celdas[1].get_text(strip=True).replace('', 'ñ').replace('', 'ó').replace('', 'é').replace('', 'á').replace('', 'í').replace('', 'ú')
            longitud_estacion = celdas[2].get_text(strip=True)
            longitud_inter = celdas[3].get_text(strip=True)
            datos.append((linea_actual, tramo, longitud_estacion, longitud_inter))
    else:
        # Continuación de la misma línea
        if len(celdas) >= 3 and linea_actual is not None:
            tramo = celdas[0].get_text(strip=True).replace('', 'ñ').replace('', 'ó').replace('', 'é').replace('', 'á').replace('', 'í').replace('', 'ú')
            longitud_estacion = celdas[1].get_text(strip=True)
            longitud_inter = celdas[2].get_text(strip=True)
            datos.append((linea_actual, tramo, longitud_estacion, longitud_inter))

with open("tramos_metro.txt", "w", encoding="utf-8") as f:
    for linea, tramo, lon_est, lon_inter in datos:
        f.write(f"Línea {linea}|{tramo}|{lon_est}|{lon_inter}\n")

In [2]:
texto = soup.get_text(strip=True, separator='\n')